**Project Coded By Nima Zare (https://nimazare.net/)**

*Project Description:*

In this project, you will build a deep learning model using TensorFlow to classify images. You'll use a popular dataset like the CIFAR-10 dataset, which contains 60,000 32x32 color images in 10 different classes. Your goal is to train a convolutional neural network (CNN) to classify these images into their respective categories.

*Resources*:

TensorFlow documentation and tutorials: https://www.tensorflow.org/

Keras documentation: https://keras.io/

CIFAR-10 dataset: https://www.cs.toronto.edu/~kriz/cifar.html

### **Import Librareis**

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG16

### **Load the Data**

In [11]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

### **Build Model**

In [12]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

58889256/58889256 [==============================] - 0s 0us/step


In [14]:
model = keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

base_model.trainable = False

In [15]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### **Train Model**

In [24]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('accuracy') is not None and logs.get('accuracy') > 0.995:
      print("\nReached 99.5% accuracy so cancelling training!")
      self.model.stop_training = True


callback = myCallback()

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=32,
                    epochs=100,
                    validation_data=(x_test, y_test),
                    callbacks=[callback],
                    verbose=2)


### **Fine-tune the model**

In [25]:
base_model.trainable = True

# Fine-tune only the last few layers
fine_tune_at = -4
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Continue training for a few more epochs
history = model.fit(x_train, y_train,
                    batch_size=20,
                    epochs=100,
                    validation_data=(x_test, y_test),
                    verbose=2,
                    callbacks=[callback])


Epoch 1/100
2500/2500 - 36s - loss: 0.1519 - accuracy: 0.9434 - val_loss: 1.4919 - val_accuracy: 0.7275 - 36s/epoch - 14ms/step
Epoch 2/100
2500/2500 - 36s - loss: 0.1499 - accuracy: 0.9446 - val_loss: 1.4903 - val_accuracy: 0.7226 - 36s/epoch - 14ms/step
Epoch 3/100
2500/2500 - 37s - loss: 0.1365 - accuracy: 0.9505 - val_loss: 1.4911 - val_accuracy: 0.7296 - 37s/epoch - 15ms/step
Epoch 4/100
2500/2500 - 37s - loss: 0.1353 - accuracy: 0.9518 - val_loss: 1.5729 - val_accuracy: 0.7223 - 37s/epoch - 15ms/step
Epoch 5/100
2500/2500 - 37s - loss: 0.1300 - accuracy: 0.9531 - val_loss: 1.6021 - val_accuracy: 0.7316 - 37s/epoch - 15ms/step
Epoch 6/100
2500/2500 - 35s - loss: 0.1267 - accuracy: 0.9545 - val_loss: 1.5384 - val_accuracy: 0.7284 - 35s/epoch - 14ms/step
Epoch 7/100
2500/2500 - 35s - loss: 0.1214 - accuracy: 0.9550 - val_loss: 1.5925 - val_accuracy: 0.7326 - 35s/epoch - 14ms/step
Epoch 8/100
2500/2500 - 35s - loss: 0.1145 - accuracy: 0.9588 - val_loss: 1.5355 - val_accuracy: 0.7266 

In [27]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_accuracy*100:.2f}%')

313/313 [==============================] - 3s 9ms/step - loss: 2.5580 - accuracy: 0.7438
Test accuracy: 74.38%


### **Save Model**

In [ ]:
model.save("cifar10_model.h5")

In [29]:
model.save('my_model.keras')

### **Test Model**

In [30]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                2570      
                                                                 
Total params: 14848586 (56.64 MB)
Trainable params: 7213322 (27.52 MB)
Non-trainable params: 7635264 (29.13 MB)
_________________________________________________________________
